In [ ]:
from hipai import paths

from aimu.models import OllamaClient
from aimu.tools import MCPClient

# Required to allow nested event loops in Jupyter notebooks (for MCPClient)
import nest_asyncio

nest_asyncio.apply()

In [ ]:
model_client = OllamaClient(OllamaClient.MODEL_MISTRAL_SMALL_3_2_24B)

MCP_SERVERS = {
    "mcpServers": {
        "hipai": {"command": "python", "args": [str(paths.package / "tools.py")]},
    }
}

mcp_client = MCPClient(MCP_SERVERS)
model_client.mcp_client = mcp_client

In [ ]:
SYSTEM_PROMPT = """
You are the user's friend and are curious and supportive if them.

Use memory tools to store and recall information about the user.
When you learn something about the user, use the `add_memories` tool to add it to memory.
Use the `search_memories` tool to search for information about the user that is relevant to the conversation.

Determine the user's name before starting the conversation.

Your name is Bruce.
"""

generate_kwargs={
    "temperature": 0.15,
    "top_p": 0.9,
    "max_new_tokens": 1024,
    "repeat_penalty": 1.1,
}

In [ ]:
model_client.messages = []

response = model_client.chat_streamed(
    {
        "role": model_client.system_role,
        "content": SYSTEM_PROMPT,
    },
    tools=mcp_client.get_tools()
)

for response_part in response:
    print(response_part, end="", flush=True)

## Memory Retrieval

In [ ]:
response = model_client.chat_streamed(
    {
        "role": "user",
        "content": "what is my name?",
    },
    tools=mcp_client.get_tools()
)

for response_part in response:
    print(response_part, end="", flush=True)

## Memory Storage

In [ ]:
response = model_client.chat_streamed(
    {
        "role": "user",
        "content": "I have a new job. I now work at a bakery.",
    },
    tools=mcp_client.get_tools()
)

for response_part in response:
    print(response_part, end="", flush=True)

In [ ]:
model_client.messages

In [ ]:
response = model_client.chat_streamed(
    {
        "role": "user",
        "content": "I am also studying at the University of Washington to learn how to develop AI models. I want to be a data scientist.",
    },
    tools=mcp_client.get_tools()
)

for response_part in response:
    print(response_part, end="", flush=True)

In [ ]:
model_client.messages